<a href="https://colab.research.google.com/github/benbaz-2/comp551/blob/main/assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3

## Installations

In [59]:
!pip install medmnist

In [82]:
import numpy as np
import matplotlib.pyplot as plt

## Load dataset

In [61]:
from medmnist import OrganAMNIST
train_dataset = OrganAMNIST(split='train', download=True)
test_dataset = OrganAMNIST(split='test', download=True)

Using downloaded and verified file: /root/.medmnist/organamnist.npz
Using downloaded and verified file: /root/.medmnist/organamnist.npz


In [62]:
# Put the data in numpy arrays
x_train, y_train = train_dataset.imgs, train_dataset.labels
x_test, y_test = test_dataset.imgs, test_dataset.labels

In [63]:
# vectorize x
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)


In [64]:
# center and normalize data
mean_train = np.mean(x_train, axis=1, keepdims=True)
std_train = np.std(x_train, axis=1, keepdims=True)
x_train = (x_train - mean_train) / std_train

mean_test = np.mean(x_test, axis=1, keepdims=True)
std_test = np.std(x_test, axis=1, keepdims=True)
x_test = (x_test - mean_test) / std_test

In [65]:
y = np.zeros((y_train.shape[0], 11))
for i in range(y.shape[0]):
  j = y_train[i]
  y[i, j] = 1
y_train_one_hot = y

y = np.zeros((y_test.shape[0], 11))
for i in range(y.shape[0]):
  j = y_test[i]
  y[i, j] = 1
y_test_one_hot = y


## Implement a MLP

In [66]:
# Activation functions

ReLU = lambda x: np.maximum(0, x)
Softmax = lambda x: np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)
cross_entropy = lambda y_true, y_pred: -np.sum(y_true * np.log(y_pred)) / y_true.shape[0]

In [67]:
# This class represents an arbitrary layer
class NeuralNetLayer:
  def __init__(self):
    self.gradient = None
    self.parameters = None

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, x):
    raise NotImplementedError()

In [74]:
# This class represents an arbitrary neural network
class DeepMLP:
  def __init__(self, layers=[], log=True):
    self.layers = layers
    if log:
      self.losses = []
    else self.losses = None

  def forward(self, x):
    for layer in self.layers:
      x = layer.forward(x)
    return x

  def backward(self, x):
    for layer in self.layers[::-1]:
      x = layer.backward(x)

  def fit(self, x, y, loss=cross_entropy, epochs=100, lr=0.01):
    for epoch in range(epochs):
      y_pred = self.forward(x)
      loss_value = loss(y, y_pred)
      dl = y_pred - y
      self.backward(dl)
      print(epoch, end=' ')
      if log:
        self.losses.append(loss_value)

      for layer in self.layers:
        if layer.parameters:  # Only update layers that have parameters
          for param in layer.parameters:
            layer.parameters[param] -= lr * layer.gradient[param]


  def predict(self, x):
    return np.argmax(self.forward(x), axis=1)


In [75]:
# Implement a linear layer

class LinearLayer(NeuralNetLayer):

  def __init__(self, input_size, output_size, log=True): # D, M
    super().__init__()
    self.input_size = input_size
    self.output_size = output_size
    self.input = None
    self.output = None

    # Use Xavier initialization
    self.parameters = {
        'weights': np.random.randn(input_size, output_size) * np.sqrt(1 / (input_size)),
        'bias': np.zeros(output_size)
    }

    self.gradient = {
        'weights': np.zeros((input_size, output_size)),
        'bias': np.zeros(output_size)
    }

  def forward(self, x):
    self.input = x  # N x D
    return np.dot(x, self.parameters['weights']) + self.parameters['bias']  # N x M

  def backward(self, dz):  # N x M
    self.gradient['weights'] = np.dot(self.input.T , dz)  # D X M
    self.gradient['bias'] = np.sum(dz, axis=0) # M
    return np.dot(dz, self.parameters['weights'].T)   # Return N x D gradient for next layer in backpropagation


In [76]:
class ReLULayer(NeuralNetLayer):
  def __init__(self):
    super().__init__()
    self.input = None
    self.output = None

  def forward(self, x):
    self.input = x  # N x D
    return ReLU(x)

  def backward(self, dz):
    return dz * (self.input > 0) # N x D Element wise multiplication

In [77]:
class SoftmaxLayer(NeuralNetLayer):
    def __init__(self):
        super().__init__()
        self.input = None
        self.output = None

    def forward(self, x):
        """
        Perform the forward pass for the softmax activation.
        Args:
            x (np.ndarray): Input data of shape (N, D) where N is the number of samples and D is the number of features.
        Returns:
            np.ndarray: Softmax probabilities of shape (N, D).
        """
        self.input = x  # Save input for backpropagation
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # Stability trick to prevent overflow
        self.output = exp_x / np.sum(exp_x, axis=1, keepdims=True)
        return self.output

    def backward(self, dz):
        """
        Perform the backward pass for the softmax activation.
        Args:
            dz (np.ndarray): Gradient of loss with respect to the output of this layer (N, D).
        Returns:
            np.ndarray: Gradient of loss with respect to the input of this layer (N, D).
        """
        N, D = dz.shape
        grad = np.zeros_like(dz)
        for i in range(N):
            s = self.output[i].reshape(-1, 1)  # (D, 1)
            jacobian = np.diagflat(s) - np.dot(s, s.T)  # (D, D)
            grad[i] = np.dot(jacobian, dz[i])  # Chain rule application
        return grad

## Model Training

In [78]:
model = DeepMLP()
model.layers.append(LinearLayer(784, 128))
model.layers.append(ReLULayer())
model.layers.append(LinearLayer(128, 11))
model.layers.append(SoftmaxLayer())

In [79]:
model.fit(x_train, y_train_one_hot)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

In [80]:
yh = model.predict(x_test)
total = len(yh)
correct = 0

for i in range(total):
  if yh[i] == y_test[i]:
    correct += 1

accuracy = correct / total
accuracy

0.08848014399820002

In [81]:
yh[0:10], y_test[0:10]

(array([ 6,  1,  8,  7,  7,  7,  1, 10,  7,  7]),
 array([[4],
        [0],
        [8],
        [6],
        [5],
        [0],
        [8],
        [7],
        [0],
        [6]], dtype=uint8))

In [81]:
plt.plot(losses)